Absolute magnitude is defined to be the apparent magnitude an object would have if it were located at a distance of 10 parsecs.
In astronomy, absolute magnitude (M) is a measure of the luminosity of a celestial object on an inverse logarithmic astronomical magnitude scale.

In [1]:
import time
import numpy as np
import h5py
import hdf5plugin
import pandas as pd
import os 
import csv
import resource

import matplotlib as mpl
import matplotlib.colors as cm
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from unyt import yr, Myr, kpc, arcsec, nJy, Mpc, Msun, erg, s, Hz
from astropy.cosmology import Planck18 as cosmo


from synthesizer.grid import Grid
from synthesizer.particle import Gas, Stars
from synthesizer.filters import Filter, FilterCollection
from synthesizer.sed import combine_list_of_seds
from synthesizer.load_data.load_camels import load_CAMELS_IllustrisTNG
from synthesizer.kernel_functions import Kernel
from synthesizer.conversions import lnu_to_absolute_mag

current_directory = os.getcwd()
print("Current directory:", current_directory)


/tmp/ipykernel_2442/3916986258.py:5: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


KeyboardInterrupt: 

In [ ]:

# Alternative method for LF:
# try this method again, but using AB mag instead of mass, and suply your own bins (up to -17, say)
def calc_lf(ab_mag, volume, massBinLimits):
# OG:        hist, dummy = np.histogram(np.log10(mstar), bins = massBinLimits)
        hist, dummy = np.histogram(ab_mag, bins = massBinLimits)
        hist = np.float64(hist)
        phi = (hist / volume) / (massBinLimits[1] - massBinLimits[0])
        phi_sigma = (np.sqrt(hist) / volume) /\
                    (massBinLimits[1] - massBinLimits[0]) # Poisson errors
        return phi, phi_sigma, hist
    

In [ ]:
grid_name = "bc03-2016-Miles_chabrier-0.1,100.hdf5"
grid_dir = "/home/jovyan/"

# Create a new grid
#grid = Grid(grid_name, grid_dir=grid_dir, read_lines=False)

# instead of using a filter, which requires us to load in large SEDs first, pass the grid wavelength
#filt1 = Filter("top_hat/filter.1", lam_min=1400, lam_max=1600, new_lam=grid.lam)
#filt_lst = [filt1]

# Define a new set of wavelengths
lims_lams=(1400, 1600)

grid = Grid(grid_name, grid_dir=grid_dir, lam_lims=lims_lams, read_lines=False)
#print(grid.shape)
#print(grid)

In [ ]:
# Define the directory where the text files will be saved
output_dir = "/home/jovyan/camels/play/synth-play/LH/output/"


In [ ]:
# broken dirs
directories = ['LH_50', 'LH_104', 'LH_177', 'LH_378', 'LH_426', 'LH_466', 'LH_513', 'LH_559', 'LH_630', 'LH_826']


In [ ]:
log_file_path = "/home/jovyan/camels/play/synth-play/LH/log_broken.txt"

# Function to log messages
def log_message(message):
    with open(log_file_path, 'a') as log_file:
        log_file.write(message + '\n')


In [ ]:

# get gals
for LH_X in directories:
    
    log_message(f"Processing directory: {LH_X}")
    
    dir_ = '/home/jovyan/Data/Sims/IllustrisTNG/LH/' + LH_X
    gals_074 = load_CAMELS_IllustrisTNG(
        dir_,
        snap_name='snapshot_074.hdf5', 
        fof_name='groups_074.hdf5',
        verbose=False,
    )

    # Filter galaxies to only include those with 100 or more star particles
    log_message(f"filtering nstars >=100")
    gals_074 = [gal for gal in gals_074 if gal.stars.nstars >= 100]
    log_message(f"filtering nparticles >=100")
    gals_074 = [gal for gal in gals_074 if gal.gas.nparticles >= 100]
    
    log_message(f"filtering age >=0")
    # Assuming gals_074 is a list or array containing multiple galaxies
    for gal in gals_074:
        # Access the Stars object for the current galaxy
        stars = gal.stars
        # Access the relevant attributes
        initial_masses = stars.initial_masses
        ages = stars.ages
        metallicities = stars.metallicities

        # Create a mask for stars with non-negative initial_masses and positive ages
        mask = (initial_masses >= 0) & (ages > 0)

        # Apply the mask to all relevant attributes
        cleaned_initial_masses = initial_masses[mask]
        cleaned_ages = ages[mask]
        cleaned_metallicities = metallicities[mask]

        # Create a new Stars object with the cleaned data
        cleaned_stars = Stars(initial_masses=cleaned_initial_masses, ages=cleaned_ages, metallicities=cleaned_metallicities)

        # Update the stars attribute of the current galaxy with the cleaned Stars object
        gal.stars = cleaned_stars

        # Optionally print the cleaned ages to verify
        #print(cleaned_stars.ages)
    
    log_message(f"finished cleaning: age, masses, npart, nstars")
        

    cat_074 = dir_+'/groups_074.hdf5'
    # open file
    f_h5py = h5py.File(cat_074, 'r')

    # read different attributes of the header
    boxSize_074 = f_h5py['Header'].attrs[u'BoxSize']/1e3 #Mpc/h
    redshift_074 = f_h5py['Header'].attrs[u'Redshift']

    spec_list = []
    for index, gal in enumerate(gals_074):
        # get_spectra_incident An Sed object containing the stellar spectra
        spec = gal.stars.get_spectra_incident(grid)
        #print(spec)
        spec.get_fnu0()
        spec_list.append(spec)


    filt1 = Filter("top_hat/filter.1", lam_min=1400, lam_max=1600, new_lam=grid.lam)
    filt_lst = [filt1]
    # combine
    log_message(f"combining SED list")
    seds = combine_list_of_seds(spec_list)
    seds.lnu # rest frame lumd
    log_message(f"getting photo lums and ABS mag")
    seds.get_photo_luminosities(filt_lst)
    seds.photo_luminosities.photo_luminosities
    abs_mag = lnu_to_absolute_mag(seds.photo_luminosities.photo_luminosities)
    abs_mag_th = abs_mag[0]

    
    # next steps, get luminosity function for these magnitudes
    # co-moving volume: BoxSize_025 and redshift:
    little_h =  0.6711
    Vphys = (boxSize_074/little_h )**3
    Vcom = Vphys * ((1+redshift_074)**3)
    
    # Perform LF calculation
    log_message(f"Performing LF calculation")
    massBinLimits = np.arange(-22, -16, 0.5)
    phi, phi_sigma, hist = calc_lf(abs_mag_th, Vcom, massBinLimits)
    massBinLimits = massBinLimits[:-1]

    output_dir = "/home/jovyan/camels/play/synth-play/LH/output/"

    # Define output file path
    output_file = f"{output_dir}{LH_X}.txt"

    # Write the data to the text file line by line
    with open(output_file, 'w') as txtfile:
        # Write phi values
        txtfile.write("phi\n")
        for value in phi:
            txtfile.write(f"{value}\n")

        # Write phi_sigma values
        txtfile.write("phi_sigma\n")
        for value in phi_sigma:
            txtfile.write(f"{value}\n")

        # Write hist values
        txtfile.write("hist\n")
        for value in hist:
            txtfile.write(f"{value}\n")

        # Write massBinLimits values
        txtfile.write("massBinLimits\n")
        for value in massBinLimits:
            txtfile.write(f"{value}\n")


    log_message(f"Written out: /home/jovyan/camels/play/synth-play/LH/output/',LH_X")
    
    # Get memory usage in bytes
    memory_usage = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
    log_message(f"Memory usage end: {memory_usage / 1000000} GB")
    log_message(f"Finished: {LH_X}")
